In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import zipfile
import shutil
from google.colab import files
import json
import time
import pandas as pd

import keras
from keras.models import Model, Sequential
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Dense, Activation, Dropout, BatchNormalization,\
                          Conv2D, MaxPooling2D, Flatten, AveragePooling2D,\
                          GlobalAveragePooling2D, ZeroPadding2D
from keras.initializers import glorot_uniform
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop, Adam, Adamax, Nadam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, \
                            classification_report

# Import PyDrive and associated libraries (to connect with GoogleDrive)
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

### **Check if we are using GPU:**

In [58]:
from keras import backend as K
if K.backend() == "tensorflow":
    import tensorflow as tf
    device_name = tf.test.gpu_device_name()
    if device_name == '':
        device_name = "None"
    print('Using TensorFlow version:', tf.__version__, ', GPU:', device_name)
    print('keras version:', keras.__version__)

Using TensorFlow version: 1.15.0 , GPU: /device:GPU:0
keras version: 2.2.5


### **Download Patches from GoogleDrive:**

In [3]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1M_CrRxfsb5y2STHQGGA5IvcUTtA6HidU' #128x128_s60_no border_minpospix_1250 minposval_1024 Downsampled Control
#file_id = '1Dwc0vZ-Atmq1-y3bQoMS_M0ReBB5-ewS' #128x128_s56_no border_minpospix_1250 minposval_1024 Downsampled Control
#file_id = '1kttFmwfFRrIVTq_ERVWV83wAi4arr1Q0' #128x128_s60_no border_minpospix_1024 minposval_1024 Downsampled Control

downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(downloaded['title'])
print('Downloaded content: "{}"'.format(downloaded['title']))
print('Root dir content: {}'.format(os.listdir()))

Downloaded content: "Patches_128_s60_min1250_minval_1024_no_border_nonNeg_noStreak_downsampledControl.zip"
Root dir content: ['.config', 'adc.json', 'Patches_128_s60_min1250_minval_1024_no_border_nonNeg_noStreak_downsampledControl.zip', 'sample_data']


### **Unzip the Patches:**

In [4]:
# Remove 'Patches' dir if it already exists
if 'Patches' in os.listdir():
  shutil.rmtree('./Patches')
with zipfile.ZipFile(downloaded['title'],"r") as zip:
    zip.extractall()
os.remove(downloaded['title'])
print('Root dir content: {}'.format(os.listdir()))

Root dir content: ['.config', 'adc.json', 'Patches', 'sample_data']


### **Let's count patches by type and location:**

In [5]:
class_weights = {} # empty dictionary to store class weights
classes = ['C1','C2-3','C4-7','C5','C6','C8','C9','C10']

grand_total, pos_total, neg_total = 0, 0, 0
for type in ['Serial', 'Control', 'Streak']:
    print("\nTotal '{}' Patches per location:".format(type))
    n_type, type_pos, type_neg = 0, 0, 0
    class_weights[type] = {} # nested empty dictionary to store class weights
    class_weights[type]['pos'] = {} # nested dictionary to store class weights
    class_weights[type]['neg'] = {} # nested dictionary to store class weights
    for cls in classes:
        pos_folder = './Patches/Positive/{}/{}_pos'.format(type,cls)
        neg_folder = './Patches/Negative/{}/{}_neg'.format(type,cls)
        n_pos = len(os.listdir(pos_folder))
        n_neg = len(os.listdir(neg_folder))
        total = n_pos + n_neg
        n_type += total
        type_pos += n_pos
        type_neg += n_neg
        print('total_{}: {} = {} positive + {} negative'.format(cls,total,n_pos,n_neg))
        class_weights[type]['pos']['{}'.format(cls)] = 1/n_pos if n_pos else 0
        class_weights[type]['neg']['{}'.format(cls)] = 1/n_neg if n_neg else 0
    print('Total {}: {} = {} positive + {} negative'.format(type,n_type,type_pos,type_neg))
    for loc in class_weights[type]['pos'].keys():
        class_weights[type]['pos'][loc] *= type_pos
    for loc in class_weights[type]['neg'].keys():
        class_weights[type]['neg'][loc] *= type_neg
    grand_total += n_type
    pos_total += type_pos
    neg_total += type_neg
print('\nGRAND TOTAL: {} = {} positive + {} negative'.format(grand_total,pos_total,neg_total))


Total 'Serial' Patches per location:
total_C1: 743 = 743 positive + 0 negative
total_C2-3: 1410 = 1410 positive + 0 negative
total_C4-7: 3057 = 3057 positive + 0 negative
total_C5: 1518 = 1518 positive + 0 negative
total_C6: 848 = 848 positive + 0 negative
total_C8: 1105 = 1105 positive + 0 negative
total_C9: 508 = 508 positive + 0 negative
total_C10: 1957 = 1957 positive + 0 negative
Total Serial: 11146 = 11146 positive + 0 negative

Total 'Control' Patches per location:
total_C1: 373 = 373 positive + 0 negative
total_C2-3: 373 = 373 positive + 0 negative
total_C4-7: 373 = 373 positive + 0 negative
total_C5: 373 = 373 positive + 0 negative
total_C6: 373 = 373 positive + 0 negative
total_C8: 373 = 373 positive + 0 negative
total_C9: 373 = 373 positive + 0 negative
total_C10: 373 = 373 positive + 0 negative
Total Control: 2984 = 2984 positive + 0 negative

Total 'Streak' Patches per location:
total_C1: 0 = 0 positive + 0 negative
total_C2-3: 0 = 0 positive + 0 negative
total_C4-7: 0 = 

#### **Since we have imbalanced training data, we have set different class weights to give more importance to the minority classes:**

In [6]:
print('Class Weights:', str(json.dumps(class_weights['Serial'], indent=2, default=str)))

Class Weights: {
  "pos": {
    "C1": 15.001345895020188,
    "C2-3": 7.904964539007093,
    "C4-7": 3.6460582270199544,
    "C5": 7.342555994729908,
    "C6": 13.143867924528301,
    "C8": 10.086877828054298,
    "C9": 21.940944881889763,
    "C10": 5.695452222789985
  },
  "neg": {
    "C1": 0,
    "C2-3": 0,
    "C4-7": 0,
    "C5": 0,
    "C6": 0,
    "C8": 0,
    "C9": 0,
    "C10": 0
  }
}


#### **Let's build image generators, using keras.preprocessing.image.ImageDataGenerator, rescaling image pixel values from [0,  255] to [0, 1]:**

In [7]:
#warnings.filterwarnings("ignore")

c1_pos_folder = './Patches/Positive/Serial/C1_pos'
img = plt.imread(c1_pos_folder + '/' + os.listdir(c1_pos_folder)[:5][0])
img_size = img.shape
train_batch_size = 32
val_batch_size = 64

# datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True,
#                              vertical_flip=True)
datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

print("For training:")
train_generator = datagen.flow_from_directory(
        './Patches/Positive/Serial',
        target_size=(img_size[0],img_size[1]),
        batch_size=train_batch_size,
        class_mode='categorical',
        shuffle=True)

print("\nFor validation:")
val_generator = val_datagen.flow_from_directory(
        './Patches/Positive/Control',
        target_size=(img_size[0],img_size[1]),
        batch_size=val_batch_size,
        class_mode='categorical',
        shuffle=False)
#warnings.filterwarnings("once")

For training:
Found 11146 images belonging to 8 classes.

For validation:
Found 2984 images belonging to 8 classes.


#### **Let's check what is the training generator's index for each class, so we can correclty set up the class weights:**

In [8]:
print('train_generator.class_indices:', str(json.dumps(train_generator.class_indices, indent=2, default=str)))
#print('validation_generator.class_indices:', str(json.dumps(val_generator.class_indices, indent=2, default=

train_generator.class_indices: {
  "C10_pos": 0,
  "C1_pos": 1,
  "C2-3_pos": 2,
  "C4-7_pos": 3,
  "C5_pos": 4,
  "C6_pos": 5,
  "C8_pos": 6,
  "C9_pos": 7
}


#### **Let's set up the class weights in correct order:**

In [9]:
serial_pos_weights = [class_weights['Serial']['pos']['C10']] # C10 has index 0
for cls in classes:
    if cls == 'C10': continue
    serial_pos_weights.append(class_weights['Serial']['pos']['{}'.format(cls)])
print('original class weights dictionary:')
print(str(json.dumps(class_weights['Serial']['pos'], indent=2, default=str)))
print('class weights for generator, re-arranging indexes:')
print(str(json.dumps(serial_pos_weights, indent=2, default=str)))

original class weights dictionary:
{
  "C1": 15.001345895020188,
  "C2-3": 7.904964539007093,
  "C4-7": 3.6460582270199544,
  "C5": 7.342555994729908,
  "C6": 13.143867924528301,
  "C8": 10.086877828054298,
  "C9": 21.940944881889763,
  "C10": 5.695452222789985
}
class weights for generator, re-arranging indexes:
[
  5.695452222789985,
  15.001345895020188,
  7.904964539007093,
  3.6460582270199544,
  7.342555994729908,
  13.143867924528301,
  10.086877828054298,
  21.940944881889763
]


### **pre-trained VGG16:**

#### **Let's build and compile our baseline model, using VGG16 pre-trained model from keras and adding trainable layers:**

In [108]:
pre_trained = VGG16(weights='imagenet', include_top=False,
                    input_shape=(128, 128, 3))

for layer in pre_trained.layers:
	layer.trainable = False

#x = Flatten()(pre_trained.output)

x = pre_trained.output
x = GlobalAveragePooling2D()(x)

predictions = Dense(n_classes, activation='softmax')(x)

base_model = Model(input=pre_trained.input, output=predictions)
base_model.classes = classes

lr = 1e-3
decay = 0.05 # 0.05
optimizer = RMSprop # (RMSprop, Adam, Adamax, Nadam)

if optimizer == keras.optimizers.Nadam:
    base_model.compile(optimizer(lr=lr, schedule_decay=decay),
                loss="categorical_crossentropy", metrics=['accuracy'])
else:
    base_model.compile(optimizer(lr=lr, decay=decay), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

base_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


### **Let's train and validate our baseline model (pre-trained VGG16):**

In [109]:
## VGG16

epochs = 50 #100

train_steps = train_generator.n//train_generator.batch_size
val_steps = val_generator.n//val_generator.batch_size

# Callbacks:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.85, patience=3, 
                                   verbose=1, mode='min', min_lr=1e-9)
EarlyStop = EarlyStopping(monitor='val_acc', patience=30, verbose=1,
                          min_delta=0, mode='max')
checkpoint = ModelCheckpoint('base_model.h5', monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max')

callbacks_list = [reduce_lr, checkpoint, EarlyStop] #order matters!

#base_model.load_weights('base_model.h5')

history = base_model.fit_generator(train_generator, steps_per_epoch=train_steps,
                            validation_data=val_generator,
                            validation_steps=val_steps, epochs=epochs,
                            verbose=1, callbacks=callbacks_list, shuffle=False,
                            class_weight=serial_pos_weights)

Epoch 1/50
348/348 [==============================] - 14s 40ms/step - loss: 1.7695 - acc: 0.4003 - val_loss: 2.0347 - val_acc: 0.2276

Epoch 00001: val_acc improved from -inf to 0.22758, saving model to base_model.h5
Epoch 2/50
348/348 [==============================] - 12s 35ms/step - loss: 1.6968 - acc: 0.4247 - val_loss: 1.9980 - val_acc: 0.2387

Epoch 00002: val_acc improved from 0.22758 to 0.23870, saving model to base_model.h5
Epoch 3/50
348/348 [==============================] - 12s 34ms/step - loss: 1.6715 - acc: 0.4300 - val_loss: 1.9846 - val_acc: 0.2432

Epoch 00003: val_acc improved from 0.23870 to 0.24315, saving model to base_model.h5
Epoch 4/50
348/348 [==============================] - 12s 34ms/step - loss: 1.6570 - acc: 0.4365 - val_loss: 1.9757 - val_acc: 0.2455

Epoch 00004: val_acc improved from 0.24315 to 0.24555, saving model to base_model.h5
Epoch 5/50
348/348 [==============================] - 12s 35ms/step - loss: 1.6437 - acc: 0.4392 - val_loss: 1.9679 - val_a

### **pre-trained VGG16 didn't work...**





### **pre-trained ResNet50:**

#### **Let's build and compile our baseline model, using Resnet50 pre-trained model from keras and adding trainable layers:**

In [111]:
classes = list(iter(train_generator.class_indices))
n_classes = len(classes)

pre_trained = ResNet50(weights='imagenet', include_top=False,
                       input_shape=(128, 128, 3))
for layer in pre_trained.layers:
	layer.trainable = False

#x = GlobalAveragePooling2D()(last)
x = Flatten()(pre_trained.output)
predictions = Dense(n_classes, activation='softmax')(x)

base_model = Model(input=pre_trained.input, output=predictions)
base_model.classes = classes

lr = 1e-3
decay = 0.05 # 0.05
optimizer = RMSprop # (RMSprop, Adam, Adamax, Nadam)

if optimizer == keras.optimizers.Nadam:
    base_model.compile(optimizer(lr=lr, schedule_decay=decay),
                loss="categorical_crossentropy", metrics=['accuracy'])
else:
    base_model.compile(optimizer(lr=lr, decay=decay), 
                       loss="categorical_crossentropy", metrics=['accuracy'])

base_model.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  del sys.path[0]


### **Let's train and validate our baseline model (pre-trained ResNet50):**

In [44]:
epochs = 50 #100

train_steps = train_generator.n//train_generator.batch_size
val_steps = val_generator.n//val_generator.batch_size

# Callbacks:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.85, patience=5, 
                                   verbose=1, mode='min', min_lr=1e-9)
EarlyStop = EarlyStopping(monitor='val_acc', patience=30, verbose=1,
                          min_delta=0, mode='max')
checkpoint = ModelCheckpoint('base_model.h5', monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max')

callbacks_list = [reduce_lr, checkpoint, EarlyStop] #order matters!

#base_model.load_weights('base_model.h5')

history = base_model.fit_generator(train_generator, steps_per_epoch=train_steps,
                            validation_data=val_generator,
                            validation_steps=val_steps, epochs=epochs,
                            verbose=1, callbacks=callbacks_list, shuffle=True)

Epoch 1/50
348/348 [==============================] - 13s 36ms/step - loss: 0.0606 - acc: 0.9779 - val_loss: 3.7564 - val_acc: 0.1267

Epoch 00001: val_acc improved from -inf to 0.12670, saving model to base_model.h5
Epoch 2/50
348/348 [==============================] - 13s 36ms/step - loss: 0.0560 - acc: 0.9803 - val_loss: 3.7308 - val_acc: 0.1277

Epoch 00002: val_acc improved from 0.12670 to 0.12774, saving model to base_model.h5
Epoch 3/50
348/348 [==============================] - 13s 36ms/step - loss: 0.0499 - acc: 0.9816 - val_loss: 3.7307 - val_acc: 0.1277

Epoch 00003: val_acc did not improve from 0.12774
Epoch 4/50
348/348 [==============================] - 13s 36ms/step - loss: 0.0512 - acc: 0.9812 - val_loss: 3.7351 - val_acc: 0.1277

Epoch 00004: val_acc did not improve from 0.12774
Epoch 5/50
348/348 [==============================] - 13s 36ms/step - loss: 0.0523 - acc: 0.9809 - val_loss: 3.7206 - val_acc: 0.1277

Epoch 00005: val_acc did not improve from 0.12774
Epoch 6/

## **pre-trained ResNet50 didn't work either....**

In [72]:
X, y_true = next(val_generator)
y_pred = base_model.predict(X)
for i in range(1, len(val_generator)):
  X, y = next(val_generator)
  y_true = np.vstack((y_true, y))
  y_pred = np.vstack((y_pred, base_model.predict(X)))

y_true = np.argmax(y_true, axis=1)
y_pred = np.argmax(y_pred, axis=1)

val_acc = accuracy_score(y_true, y_pred)
#roc_auc = roc_auc_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
class_names = [k for k in val_generator.class_indices]
c_report = classification_report(y_true, y_pred, target_names=class_names)

print('\nval_acc:\n', val_acc)
print('\nConfusion Matrix:\n', cm)
print('\nClassification Report:\n', c_report)


val_acc:
 0.12935656836461126

Confusion Matrix:
 [[ 13   0   0 360   0   0   0   0]
 [  0   0   0 373   0   0   0   0]
 [  0   0   0 373   0   0   0   0]
 [  0   0   0 373   0   0   0   0]
 [  0   0   0 373   0   0   0   0]
 [  0   0   0 373   0   0   0   0]
 [  0   0   0 373   0   0   0   0]
 [  0   0   0 373   0   0   0   0]]

Classification Report:
               precision    recall  f1-score   support

     C10_pos       1.00      0.03      0.07       373
      C1_pos       0.00      0.00      0.00       373
    C2-3_pos       0.00      0.00      0.00       373
    C4-7_pos       0.13      1.00      0.22       373
      C5_pos       0.00      0.00      0.00       373
      C6_pos       0.00      0.00      0.00       373
      C8_pos       0.00      0.00      0.00       373
      C9_pos       0.00      0.00      0.00       373

    accuracy                           0.13      2984
   macro avg       0.14      0.13      0.04      2984
weighted avg       0.14      0.13      0.04    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
